In [3]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [5]:
data_train = pd.read_csv('C:\\Study\\Kaggle-Stuff\\Kaggle-MegaFon-MGTU\\contest_train_binary.csv')
target_train = data_train['TARGET']
del data_train['TARGET']
del data_train['ID']
data_test = pd.read_csv('C:\\Study\\Kaggle-Stuff\\Kaggle-MegaFon-MGTU\\contest_test.csv')
del data_test['ID']
# Для удобства имена всех фичей
temp = []
for i in data_test.columns:
    temp.append(i)
    #print(i)
names = pd.Series(temp, index=temp)
print(names)

FEATURE_0        FEATURE_0
FEATURE_1        FEATURE_1
FEATURE_2        FEATURE_2
FEATURE_3        FEATURE_3
FEATURE_4        FEATURE_4
                  ...     
FEATURE_255    FEATURE_255
FEATURE_256    FEATURE_256
FEATURE_257    FEATURE_257
FEATURE_258    FEATURE_258
FEATURE_259    FEATURE_259
Length: 260, dtype: object


In [7]:
# Смотрим кол-ва уникальных значений всех признаков
n_unique = data_train.nunique()
print(n_unique.sort_values(ascending=False))
#print(n_unique.sort_values()[30:90])

FEATURE_55     24521
FEATURE_63     24521
FEATURE_147    24521
FEATURE_79     24521
FEATURE_78     24521
               ...  
FEATURE_15         2
FEATURE_144        1
FEATURE_256        1
FEATURE_3          1
FEATURE_249        1
Length: 260, dtype: int64


In [11]:
# Есть признаки, в которых вообще только одно уникальное значение
# И в них при этом нет миссингов, так что они реально бесполезные
useless_features = n_unique[n_unique < 2]
print(useless_features.size)

# Смотрим, какие признаки бинарные
binary_features = n_unique[n_unique == 2]
print(binary_features.size)
binary_features_names = names[n_unique == 2]
#print(binary_features_names)


4
20


In [45]:
# Точно числовыми будем считать те, где количество уникальных значений больше 1000
numerical_features_names1 = names[n_unique > 1000]
print(numerical_features_names.size)

117


In [46]:
# Остальные возможно категориальные
mb_categorial_names = names[(n_unique > 2) & (n_unique <= 1000)]
print(mb_categorial_names.size)


119


In [47]:
# Сначала в числовые запишем те, где в уникальных значениях десятичные части не нулевые
# А также те, где есть отрицательные уникальные значения
temp = []
for name_i in mb_categorial_names:
    if np.nansum(data_train[name_i].unique() % 1) > 0:
        temp.append(name_i)
    elif np.nansum(data_train[name_i].unique() < 0):
        #print(data_train[name_i].unique())
        temp.append(name_i)
numerical_features_names2 = names[temp]
print(numerical_features_names2.size)

19


c:\users\sadbattlecruiser\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in remainder
  """
c:\users\sadbattlecruiser\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in less
  import sys


In [48]:
# Из оставшихся возможно категориальных
mb_categorial_names2 = mb_categorial_names[~(mb_categorial_names.isin(numerical_features_names2))]
print(mb_categorial_names2)

FEATURE_0        FEATURE_0
FEATURE_1        FEATURE_1
FEATURE_9        FEATURE_9
FEATURE_10      FEATURE_10
FEATURE_13      FEATURE_13
                  ...     
FEATURE_247    FEATURE_247
FEATURE_248    FEATURE_248
FEATURE_257    FEATURE_257
FEATURE_258    FEATURE_258
FEATURE_259    FEATURE_259
Length: 100, dtype: object


In [51]:
# Надо что-то сделать с оставшимися 
for name_i in mb_categorial_names2:
    print('-------------------------')
    print(data_train[name_i].value_counts())

-------------------------
0.0      10612
1.0       3158
2.0       2023
3.0       1397
4.0       1101
         ...  
134.0        1
379.0        1
156.0        1
197.0        1
270.0        1
Name: FEATURE_0, Length: 160, dtype: int64
-------------------------
0.0     10612
1.0      5923
2.0      3451
3.0      1903
4.0      1103
5.0       648
6.0       376
7.0       202
8.0       116
9.0        76
10.0       50
12.0       19
11.0       16
13.0       10
14.0        7
19.0        2
17.0        2
31.0        1
15.0        1
39.0        1
18.0        1
30.0        1
Name: FEATURE_1, dtype: int64
-------------------------
0.0     19643
1.0      1846
2.0      1743
3.0      1141
4.0       124
5.0        14
7.0         4
11.0        2
8.0         2
6.0         2
Name: FEATURE_9, dtype: int64
-------------------------
0.0    19643
1.0     3942
2.0      787
3.0      128
4.0       14
5.0        7
Name: FEATURE_10, dtype: int64
-------------------------
0.0      6036
1.0      5714
2.0      3650
3.0

Name: FEATURE_194, Length: 570, dtype: int64
-------------------------
0.0       7193
1.0       1598
2.0       1508
3.0       1037
4.0        741
          ... 
1350.0       1
634.0        1
434.0        1
1429.0       1
1082.0       1
Name: FEATURE_196, Length: 977, dtype: int64
-------------------------
0.0      19163
1.0       2179
2.0       1060
3.0        486
4.0        254
         ...  
170.0        1
451.0        1
93.0         1
115.0        1
148.0        1
Name: FEATURE_197, Length: 130, dtype: int64
-------------------------
0.0     20293
1.0      1780
2.0       912
3.0       362
4.0       210
        ...  
61.0        1
72.0        1
82.0        1
69.0        1
71.0        1
Name: FEATURE_198, Length: 67, dtype: int64
-------------------------
0.0     23396
1.0       475
2.0       228
3.0       108
4.0        67
5.0        47
6.0        31
8.0        21
7.0        18
9.0        14
11.0       11
15.0       10
10.0        8
14.0        8
13.0        8
16.0        6
18.0     